# Setup

additional information: https://github.com/niessner/Matterport/blob/master/data_organization.md 

In [11]:
import sys
import subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "open3d"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "imageio"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "pillow"])


0

In [12]:
# # if you want to use GDrive, uncommpaned this...
# from google.colab import drive
# drive.mount('/content/drive')

### File Locations

In [13]:
camera_transforms_path = r"G:/My Drive/GSD_AI/02_Models/Matterport/Matterport_Scans_Imgs/scans/17DRP5sb8fy/undistorted_camera_parameters/17DRP5sb8fy/undistorted_camera_parameters/17DRP5sb8fy.conf"
meshes_path = r"G:/My Drive/GSD_AI/02_Models/Matterport/Matterport_Scans_Imgs/scans/17DRP5sb8fy/matterport_mesh/17DRP5sb8fy/_/matterport_mesh/bed1a77d92d64f5cbbaaae4feed64ec1/bed1a77d92d64f5cbbaaae4feed64ec1.obj"
depth_imgs_dir = r"G:/My Drive//GSD_AI/02_Models/Matterport/Matterport_Scans_Imgs/scans/17DRP5sb8fy/undistorted_depth_images/17DRP5sb8fy/_/undistorted_depth_images"
color_imgs_dir = r"G:/My Drive//GSD_AI/02_Models/Matterport/Matterport_Scans_Imgs/scans/17DRP5sb8fy/undistorted_color_images/17DRP5sb8fy/_/undistorted_color_images"
mask_imgs_dir = r"G:/My Drive/GSD_AI/05_Segmentation/17DRP5sb8fy/SAM"
segmentation_data_csv_path = r"G:/My Drive/GSD_AI/05_Segmentation/17DRP5sb8fy/SAM/ImageClassifications_collected.csv"

### Settings

In [ ]:
img_width = 1280
img_height = 1024
scale_factor = 4200
img_sample_stride = 10 #density of samples of depth maps, denser=slower but more accurate
dist_threshold = 0.5 #max dist of raycast point to mesh
# allowable_materials = {'attached':'#5ECFDB', 
#                        'loose':'#BADB5E'}
allowable_materials = {'concrete':'#5ECFDB', 
                       'metal':'#BADB5E', 
                       'wood':"#BB00FF", 
                       'brick':'#5C5550', 
                       'glass':'#595C50', 
                       'quartz': '#505B5C', 
                       'granite': '#6F6175',
                       'stone':'#9C7F6A', 
                       'gypsum':'#8E9C6A',
                       'plastic': '#B65EDB',
                       'tile':'#DB935E',
                       'carpet':'#966AA8',
                       'other':'#6A979C'}

### Load Data

In [ ]:
import open3d as o3d
import open3d.core as o3c
from open3d.t.geometry import RaycastingScene
import numpy as np
import os
import re
import glob
from PIL import ImageColor
# import matplotlib.pyplot as plt
# from mpl_toolkits import mplot3d
import imageio.v3 as iio


In [16]:
raw_scan = o3d.io.read_triangle_mesh(meshes_path)
raw_scan.paint_uniform_color((0.25,0.25,0.25))
#setup raycast scene
ray_cast_scene = o3d.t.geometry.RaycastingScene()
geom_id_to_mesh = {}

full_mesh = o3d.t.geometry.TriangleMesh.from_legacy(raw_scan)
geom_id = ray_cast_scene.add_triangles(full_mesh)
print(geom_id)

#geom_id_to_mesh[geom_id] = geom_id

[Open3D WARNING] Legacy mesh has more than 1 material which is not supported by Tensor-based mesh. Only material DefaultMaterial was converted.
0


#### Check That Everything Imported
Note that this will not work in Colab, only in jupyter notebooks running on your own setup.

In [17]:
o3d.visualization.draw_geometries([raw_scan])

#### Load Camera Transforms and Data

In [18]:
class camera_pose_data:

  camera_transform = np.empty(0)
  depth_img = ""
  color_img = ""
  focal_len_x_pixels, focal_len_y_pixels, camera_center_x_pixels, camera_center_y_pixels = 0, 0, 0, 0
  camera_position = np.array([[0.0, 0.0, 0.0]])
  mask_imgs = []
  def __init__(self, depth_img, color_img, camera_transform, focal_len_x_pixels, focal_len_y_pixels, camera_center_x_pixels, camera_center_y_pixels):
    self.depth_img = depth_img
    self.color_img = color_img
    self.camera_transform = camera_transform
    self.focal_len_x_pixels = focal_len_x_pixels
    self.focal_len_y_pixels = focal_len_y_pixels
    self.camera_center_x_pixels = camera_center_x_pixels
    self.camera_center_y_pixels = camera_center_y_pixels
    return


In [19]:
camera_poses = {}
try:
  with open(camera_transforms_path, 'r') as file:
    focal_len_x_pixels, focal_len_y_pixels, camera_center_x_pixels, camera_center_y_pixels = 0, 0, 0, 0
    for i, line in enumerate(file):
      clean_line = line.strip()
      sections = clean_line.split(' ')
      cleaned_sections = []
      for s in sections:
        if (s != ''):
          cleaned_sections.append(s)
      sections = cleaned_sections

      ct = len(sections)
      if (ct < 10):
        continue
      if (sections[0] == "intrinsics_matrix" and ct == 10):
        focal_len_x_pixels = float(sections[1])
        focal_len_y_pixels = float(sections[5])
        camera_center_x_pixels = float(sections[3])
        camera_center_y_pixels = float(sections[6])
      if (sections[0] != "scan"):
        continue
      if (ct != 19):
        print(f"ERROR: Unexpected number of sections in line {i}")
      depth_img_path = depth_imgs_dir + "/" + sections[1]
      rgb_img_path = color_imgs_dir + "/" + sections[2]

      if (not os.path.exists(depth_img_path) or not os.path.exists(rgb_img_path)):
        print(f"ERROR: File not found: {depth_img_path} {rgb_img_path}")
        continue
      t = np.eye(4)#empty 4x4 transform matrix
      for r in range(0, 4):
        for c in range (0, 4):
          ray_idx = 3 + (r * 4) + c
          t[r, c] = float(sections[ray_idx])

      #rotate 180degrees around x axis--it was found that doing this extra transform was necessary rather than relying on the imported transform by itself
      #upper left 3x3 part of the 4x4 transform matrix is the rotation matrix
      #this gives us rotate_x = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]
      #matrix for rotating about x-axis: Rx(t)= [[1, 0, 0], [0, cos(t), -sin(t)], [0, sin(t), cos(t)]]
      rotate_x = np.eye(4)
      rotate_x[1,1] = -1
      rotate_x[2,2] = -1
      t = t @ rotate_x

      pose = camera_pose_data(depth_img_path, rgb_img_path, t, focal_len_x_pixels, focal_len_y_pixels, camera_center_x_pixels, camera_center_y_pixels)

      #get image key
      sections = re.split(r'[/.]', rgb_img_path)
      key = sections[len(sections) - 2]
      #print(f"image name: {key}")
      if not key in camera_poses:
        camera_poses[key] = pose
      else:
        print(f"key already found in dictionary: {key}")
      




except FileNotFoundError:
  print(f"File not found: {camera_transforms_path}")

print(f"successfully imported {len(camera_poses)} poses")

successfully imported 864 poses


In [33]:
print(camera_poses[next(iter(camera_poses))].camera_transform) #check that transform values came in properly

[[-0.73677    0.438446   0.514718   3.52064  ]
 [-0.675941  -0.496262  -0.544819  -0.501891 ]
 [ 0.0165615 -0.749326   0.661995   1.54282  ]
 [ 0.         0.         0.         1.       ]]


# Process Depth Images

In [21]:
#check that file loads and has a valid type
#images are uint16 pngs where each integer increment = 0.25mm (/4000 to get meters)

img = o3d.io.read_image(camera_poses[next(iter(camera_poses))].depth_img)
print("dtype:", np.asarray(img).dtype)
print("min/max:", np.min(np.asarray(img)), np.max(np.asarray(img)))


dtype: uint16
min/max: 0 6922


In [22]:
def depth_to_point_cloud(depth_img, stride, intrinsic, scale):
    depth_np = np.asarray(depth_img).astype(np.float32) / float(scale)
    h, w = depth_np.shape
    fx, fy = intrinsic.get_focal_length()
    cx, cy = intrinsic.get_principal_point()

    # Create meshgrid of pixel coordinates
    u, v = np.meshgrid(np.arange(0, w, stride), np.arange(0, h, stride))
    z = depth_np[::stride, ::stride]
    x = (u - cx) * z / fx
    y = (v - cy) * z / fy

    points = np.stack((x, y, z), axis=-1).reshape(-1, 3)
    pixel_coords = np.stack((u, v), axis=-1).reshape(-1, 2)
    validity_map = []
    for p in points:
      if (p[0] == 0 and p[1] == 0  and p[2] == 0 ):
        validity_map.append(False)
      else:
        validity_map.append(True)
    points = points[validity_map]

    return points, validity_map, pixel_coords

depth_clouds = {}
validity_maps = {}
pixel_maps = {}
camera_positions = []
ct = len(camera_poses)
i = 0
for key in camera_poses:
  pose = camera_poses[key]
  img = o3d.io.read_image(pose.depth_img)
  print(pose.depth_img)
  #img = iio.imread(pose.depth_img)
  intrinsic = o3d.camera.PinholeCameraIntrinsic(width = img_width, height = img_height, fx = pose.focal_len_x_pixels, fy = pose.focal_len_y_pixels, cx = pose.camera_center_x_pixels, cy = pose.camera_center_y_pixels)
  #o3d was crashing when applying project_valid_depth_only = False and stride > 1. We need information for each sample, not just valid ones.
  cloud_pts, validity_map, pixel_coords = depth_to_point_cloud(img, img_sample_stride, intrinsic, scale_factor)
  validity_maps[key] = validity_map
  pixel_maps[key] = pixel_coords
  cloud = o3d.geometry.PointCloud()
  cloud.points = o3d.utility.Vector3dVector(cloud_pts)

  # don't do this:
  # cloud = o3d.geometry.PointCloud.create_from_depth_image(img,
  #                                                               intrinsic,
  #                                                               depth_scale = scale_factor,
  #                                                               depth_trunc = 50,
  #                                                               stride = img_sample_stride,
  #                                                               project_valid_depth_only = False)

  # Get Camera Position transformed
  camera_position = cam_pos_world = pose.camera_transform[:3, 3]
  pose.camera_position = camera_position
  camera_positions.append(camera_position)

  #transform the depth cloud
  cloud.transform(pose.camera_transform)
  depth_clouds[key] = cloud
  print(f"cloud size: {len(cloud.points)}")
  print(f"{str(i + 1)} of {str(ct)}")
  i += 1
  # if (i > 50):
  #   break



G:/My Drive//GSD_AI/02_Models/Matterport/Matterport_Scans_Imgs/scans/17DRP5sb8fy/undistorted_depth_images/17DRP5sb8fy/_/undistorted_depth_images/00ebbf3782c64d74aaf7dd39cd561175_d0_0.png
cloud size: 12255
1 of 864
G:/My Drive//GSD_AI/02_Models/Matterport/Matterport_Scans_Imgs/scans/17DRP5sb8fy/undistorted_depth_images/17DRP5sb8fy/_/undistorted_depth_images/00ebbf3782c64d74aaf7dd39cd561175_d0_1.png
cloud size: 11994
2 of 864
G:/My Drive//GSD_AI/02_Models/Matterport/Matterport_Scans_Imgs/scans/17DRP5sb8fy/undistorted_depth_images/17DRP5sb8fy/_/undistorted_depth_images/00ebbf3782c64d74aaf7dd39cd561175_d0_2.png
cloud size: 12032
3 of 864
G:/My Drive//GSD_AI/02_Models/Matterport/Matterport_Scans_Imgs/scans/17DRP5sb8fy/undistorted_depth_images/17DRP5sb8fy/_/undistorted_depth_images/00ebbf3782c64d74aaf7dd39cd561175_d0_3.png
cloud size: 11854
4 of 864
G:/My Drive//GSD_AI/02_Models/Matterport/Matterport_Scans_Imgs/scans/17DRP5sb8fy/undistorted_depth_images/17DRP5sb8fy/_/undistorted_depth_images

## Verify depth clouds match model.
You can look at a specific image and visually-inspect if the point cloud is aligning properly with the model. Some minor deviation and random noise is expected.

In [ ]:
clds = []
for cloud in depth_clouds:
    clds.append(depth_clouds[cloud])
o3d.visualization.draw_geometries([raw_scan, clds[20]])

TypeError: draw_geometries(): incompatible function arguments. The following argument types are supported:
    1. (geometry_list: list[open3d.cpu.pybind.geometry.Geometry], window_name: str = 'Open3D', width: int = 1920, height: int = 1080, left: int = 50, top: int = 50, point_show_normal: bool = False, mesh_show_wireframe: bool = False, mesh_show_back_face: bool = False, lookat: Optional[numpy.ndarray[numpy.float64[3, 1]]] = None, up: Optional[numpy.ndarray[numpy.float64[3, 1]]] = None, front: Optional[numpy.ndarray[numpy.float64[3, 1]]] = None, zoom: Optional[float] = None) -> None

Invoked with: [TriangleMesh with 191158 points and 215757 triangles., [PointCloud with 12255 points., PointCloud with 11994 points., PointCloud with 12032 points., PointCloud with 11854 points., PointCloud with 11665 points., PointCloud with 12007 points., PointCloud with 12853 points., PointCloud with 12585 points., PointCloud with 11910 points., PointCloud with 12765 points., PointCloud with 12987 point

# Denoise Point Clouds

Check each point cloud point and make sure it is close to mesh. If not, remove it.

In [24]:
def euclidean_distance(point1, point2):
    "Calculates the Euclidean distance between two 3D points."
    point1 = np.asarray(point1)
    point2 = np.asarray(point2)
    return np.linalg.norm(point1 - point2)


face_id_maps = {}
#geometry_id_maps = {}

ct = len(depth_clouds)
mod_validity_maps = validity_maps.copy()
cloudpts = []
i = -1
for key in depth_clouds:
  i += 1
  
  cloud = depth_clouds[key]
  v_map = mod_validity_maps[key]
  pose = camera_poses[key]

  ray_sources = []
  ray_targets = []
  map_size = len(v_map)
  local_valid_map = v_map.copy()
  local_triangle_ids = np.full(map_size, -1)

  #get ray sources/targets
  #this needs to be limited to valid parts of the depth image, to get a ray that is valid
  position = pose.camera_position
  valid_ct = 0
  print(f"pt ct: {len(cloud.points)}")
  cloudIdx = 0
  for ray_idx, v in enumerate(local_valid_map):
    if not local_valid_map[ray_idx]:
      continue
    point = cloud.points[cloudIdx]
    cloudIdx += 1
    ray_sources.append(position)
    ray_targets.append(point)
    cloudpts.append(point)
    valid_ct += 1
  print(f"starting valid ct: {valid_ct}")

  #normalize the rays
  print(f"ray ct: {len(ray_sources)}")
  rays_sources = np.array(ray_sources)
  rays_targets = np.array(ray_targets)
  directions = ray_targets - rays_sources
  norms = np.linalg.norm(directions, axis=1, keepdims= True)
  directions_normalized = directions / norms
  rays = o3c.Tensor(np.hstack((rays_sources, directions_normalized)), dtype=o3c.Dtype.Float32)

  #compute
  results = ray_cast_scene.cast_rays(rays)
  hit_points = ray_sources + directions_normalized * results["t_hit"].numpy()[..., None]

  #bring values from shortened "valid" ray list back to the full array
  map_idx = 0
  for ray_idx, ids in enumerate(results['primitive_ids']):
    id = ids.item()
    if  id == RaycastingScene.INVALID_ID:
      continue
    hp = hit_points[ray_idx]
    dist = euclidean_distance(ray_targets[ray_idx], hp)
    if (dist > dist_threshold):
      continue


    local_valid_map[map_idx] = True
    local_triangle_ids[map_idx] = id
    map_idx += 1

  face_id_maps[key] = local_triangle_ids

  mod_validity_maps[key] = local_valid_map
  print(f"ending valid ct: {valid_ct}")
  print(len(results['primitive_ids']))
  print(f"{str(i + 1)} of {str(ct)}")






pt ct: 12255
starting valid ct: 12255
ray ct: 12255
ending valid ct: 12255
12255
1 of 864
pt ct: 11994
starting valid ct: 11994
ray ct: 11994
ending valid ct: 11994
11994
2 of 864
pt ct: 12032
starting valid ct: 12032
ray ct: 12032
ending valid ct: 12032
12032
3 of 864
pt ct: 11854
starting valid ct: 11854
ray ct: 11854
ending valid ct: 11854
11854
4 of 864
pt ct: 11665
starting valid ct: 11665
ray ct: 11665
ending valid ct: 11665
11665
5 of 864
pt ct: 12007
starting valid ct: 12007
ray ct: 12007
ending valid ct: 12007
12007
6 of 864
pt ct: 12853
starting valid ct: 12853
ray ct: 12853
ending valid ct: 12853
12853
7 of 864
pt ct: 12585
starting valid ct: 12585
ray ct: 12585
ending valid ct: 12585
12585
8 of 864
pt ct: 11910
starting valid ct: 11910
ray ct: 11910
ending valid ct: 11910
11910
9 of 864
pt ct: 12765
starting valid ct: 12765
ray ct: 12765
ending valid ct: 12765
12765
10 of 864
pt ct: 12987
starting valid ct: 12987
ray ct: 12987
ending valid ct: 12987
12987
11 of 864
pt ct: 1

# Apply Segmentation Results to Model

In [ ]:
face_id_materials = np.zeros((len(full_mesh.triangle.indices), len(allowable_materials)), dtype=np.int32)
print(f"size of face id to materials array: {face_id_materials.shape}")
mask_ct = 0
mat_list = []
for mat in allowable_materials:
  mat_list.append(mat)

try:
  pattern = "*.png"
  with open(segmentation_data_csv_path, 'r') as file:
    lines = file.readlines()
    ct = len(lines)
    for i, line in enumerate(lines):
      line = line.strip()
      sections = line.split(',')
      orig_path = sections[0]
      #desc = sections[1]
      #fam = sections[2]
      #cat = sections[3]
      mat = sections[4]
      
      sections = re.split(r'[/.]', orig_path)
      mask_id = sections[len(sections) - 2]
      key = sections[len(sections) - 4]
      
      try:
        pose = camera_poses[key]
        v_map = validity_maps[key]
        pixel_coords = pixel_maps[key]
        face_ids = face_id_maps[key]
        #geom_ids = geometry_id_maps[key]
      except KeyError:
        print(f"{key}  not found in dictionary. Are you processing a subset of frames?")
        continue

      #cull out hallucinated material tags
      if not mat in mat_list: 
        print(f"{mat} is not an accepted material and is being excluded")
        continue

      mat_id = mat_list.index(mat, 0, len(mat_list))

      mask_path = f"{mask_imgs_dir}/{key}/masks/{mask_id}.png"
      if not os.path.exists(mask_path):
        print(f"mask image not found: {mask_path}")
        continue

      img = iio.imread(mask_path)

      valid_indices = np.where(v_map)[0]
      coords = pixel_coords[valid_indices]
      face_ids_valid = face_ids[valid_indices]
      pix_vals = img[coords[:, 1], coords[:, 0]]
      mask_hits = (pix_vals != 0)
      selected_face_ids = face_ids_valid[mask_hits]
      np.add.at(face_id_materials[:, mat_id], selected_face_ids, 1)

      # rows = pixel_coords[:, 1].astype(int)  # v → y
      # cols = pixel_coords[:, 0].astype(int)  # u → x


      # ray_idx = 0
      # for valid in v_map:
      #   if not valid:
      #     continue
      #   #get pixel value at this coordinate in the mask image
      #   #coord = pixel_coords[ray_idx]
      #   pix_val = img[rows[ray_idx],cols[ray_idx]]
      #   #print(pix_val)
      #   #determine if this pixel is inside a (non-black) masked region
      #   if (pix_val == 0):
      #     continue
      #   f_id = face_ids[ray_idx]
      #   face_id_materials[f_id][mat_id] += 1
      #   ray_idx += 1
      #   #g_id = valid_geom_ids[i]

      
      print(f"Processed {i} of {ct}")

      mask_ct += 1

      
      
      

except FileNotFoundError:
  print(f"File not found: {camera_transforms_path}")

print(f"successfully processed {mask_ct} masks")

size of face id to materials array: (215757, 2)
Processed 0 of 10428
Processed 1 of 10428
Processed 2 of 10428
Processed 3 of 10428
Processed 4 of 10428
Processed 5 of 10428
Processed 6 of 10428
Processed 7 of 10428
Processed 8 of 10428
Processed 9 of 10428
 attached  is not an accepted material and is being excluded
Processed 11 of 10428
Processed 12 of 10428
Processed 13 of 10428
Processed 14 of 10428
Processed 15 of 10428
Processed 16 of 10428
Processed 17 of 10428
Processed 18 of 10428
Processed 19 of 10428
Processed 20 of 10428
Processed 21 of 10428
Processed 22 of 10428
Processed 23 of 10428
Processed 24 of 10428
Processed 25 of 10428
Processed 26 of 10428
Processed 27 of 10428
Processed 28 of 10428
Processed 29 of 10428
Processed 30 of 10428
Processed 31 of 10428
Processed 32 of 10428
Processed 33 of 10428
Processed 34 of 10428
Processed 35 of 10428
Processed 36 of 10428
Processed 37 of 10428
Processed 38 of 10428
Processed 39 of 10428
Processed 40 of 10428
Processed 41 of 10428

## Decide vote winners
For each mesh face, the material with the most votes wins.

In [60]:
winners = []
colors = []
for face_tallies in face_id_materials:
    max_idx = np.argmax(face_tallies)
    w = mat_list[max_idx]
    winners.append(w)
    colors.append(np.array(ImageColor.getcolor(allowable_materials[w], "RGB"))/255.0)
print("winners determined")

winners determined


# Visualize

In [ ]:
triangles = full_mesh.triangle["indices"].numpy()
vertices = full_mesh.vertex["positions"].numpy()

new_vertices = []
new_triangles = []
new_colors = []
for i, tri in enumerate(triangles):
    tri_verts = vertices[tri]
    new_vertices.extend(tri_verts)
    new_triangles.append([3*i, 3*i+1, 3*i+2])
    c = colors[i]
    new_colors.extend([c, c, c])


# Step 2: Create new mesh
colored_mesh = o3d.geometry.TriangleMesh()
colored_mesh.vertices = o3d.utility.Vector3dVector(new_vertices)
colored_mesh.triangles = o3d.utility.Vector3iVector(new_triangles)
colored_mesh.vertex_colors = o3d.utility.Vector3dVector(new_colors)

print("Vertices:", len(colored_mesh.vertices))
print("Triangles:", len(colored_mesh.triangles))
print("Has vertex colors:", colored_mesh.has_vertex_colors())
print("Bounding box:", colored_mesh.get_axis_aligned_bounding_box())

print("Has vertex colors:", colored_mesh.has_vertex_colors())
print("First color:", np.asarray(colored_mesh.vertex_colors)[0])

# Optional: compute normals for better rendering
colored_mesh.compute_vertex_normals()
colored_mesh.compute_triangle_normals()

# output file
o3d.io.write_triangle_mesh("C:/Repos/GL-Material-Classification/dbg.ply", colored_mesh)

# Step 3: Visualize
o3d.visualization.draw_geometries(
    [colored_mesh],
    zoom=0.7,
    front=[0, 0, -1],
    lookat=colored_mesh.get_center(),
    up=[0, -1, 0]
)




Vertices: 647271
Triangles: 215757
Has vertex colors: True
Bounding box: AxisAlignedBoundingBox: min: (-11.5934, -2.88662, -0.127553), max: (4.75703, 5.39202, 2.67878)
Has vertex colors: True
First color: [0.36862745 0.81176471 0.85882353]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
